In [ ]:
import pandas as pd
import pickle
recipedf = pickle.load(open("finalrecipedf.p","rb"))

In [ ]:
recipedf = recipedf.reset_index()
recipedf = recipedf.drop("index",axis=1)

In [ ]:
recipedf

# Putting Total Time as (Days, Hours, Minutes)

In [ ]:
recipedf["PrepTime"] = recipedf['PrepTime'].str.strip()
recipedf["CookTime"] = recipedf['CookTime'].str.strip()

In [ ]:
#Cleaning PrepTime
for i in range(len(recipedf)):
    k = recipedf.iloc[i]["PrepTime"]
    t = None
    if(k == "3-Above"): #3above
        t = (3,0,0)
        
    if(k == "Preparation Time"): #notimegiven
        t = (0,0,0)
    
    if "days" in k: #days
        t = (int(k.split("-")[0]),0,0)
    
    if "hour" in k: #hours
        first = k.split("-")[0]
        if '.' in first:
            h = first.split(".")[0]
            m = first.split(".")[1]
            t = (0,int(h),int(m))
        else:
            t = (0,int(first),0)
            
    if "minutes" in k: #minutes
        t = (0,0,int(k.split("-")[0]))
    
    recipedf.iloc[i]["PrepTime"] = t
        

In [ ]:
#Cleaning CookTime
for i in range(len(recipedf)):
    k = recipedf.iloc[i]["CookTime"]
    t = None
    if(k == "3-Above"): #3above
        t = (3,0,0)
        
    if(k == "Cooking Time"): #notimegiven
        t = (0,0,0)
    
    if "days" in k: #days
        t = (int(k.split("-")[0]),0,0)
    
    if "hour" in k: #hours
        first = k.split("-")[0]
        if '.' in first:
            h = first.split(".")[0]
            m = first.split(".")[1]
            t = (0,int(h),int(m))
        else:
            t = (0,int(first),0)
            
    if "minutes" in k:
        t = (0,0,int(k.split("-")[0]))
    
    recipedf.iloc[i]["CookTime"] = t
        

In [ ]:
#Making total time 
recipedf["Total Time"] = None
totaltime = []
for i in range(len(recipedf)):
    m = recipedf.iloc[i]["Total Time"]
    p = list(recipedf.iloc[i]["PrepTime"])
    c = list(recipedf.iloc[i]["CookTime"])
    t = [p[k] + c[k] for k in range(3)]
    
    if(t[2]>=60):
        t[1] += t[2]//60
        t[2] = t[2]%60
    
    if(t[1]>=24):
        t[0] += t[1]//24
        t[1] = t[1]%24
    
    t = tuple(t)
    
    totaltime.append(t)

totaltimeseries = pd.Series(totaltime)
recipedf["Total Time"] = totaltimeseries

In [ ]:
#dropping prep and cook time
recipedf = recipedf.drop(["PrepTime","CookTime"],axis=1)
recipedf = recipedf[["URL","Dish","Cuisine","Total Time","Ingredients","NV"]]

In [ ]:
recipedf 

# Cleaning Dish Names and Adding More Categories

In [ ]:
#removing "Recipe" or "Recipe Card" from Dish name
recipedf['Dish'] = recipedf['Dish'].str.replace("Recipe Card","")
recipedf['Dish'] = recipedf['Dish'].str.replace("Recipe","")
recipedf['Dish'] = recipedf['Dish'].str.strip()

In [ ]:
#changing column name NV to Style
recipedf = recipedf.rename(columns={'NV':'Style'})

In [ ]:
recipedf

In [ ]:
#including Vegan,Satvik and Gluten Free tags
for i in range(len(recipedf)):
    if "Vegan" in recipedf.iloc[i]["Dish"]:
        recipedf.iloc[i]["Style"] = "Vegan"
    if "Satvik" in recipedf.iloc[i]["Dish"] or "Jain" in recipedf.iloc[i]["Dish"]:
        recipedf.iloc[i]["Style"] = "Satvik"
    if "Gluten Free" in recipedf.iloc[i]["Dish"]:
        recipedf.iloc[i]["Style"] = "Gluten Free"
    

In [ ]:
recipedf[recipedf["Style"] == "Satvik"]

In [ ]:
recipedf.to_excel("recipedfsemiclean.xlsx")

In [ ]:
recipedf

# Cuisine Cleaning

In [ ]:
recipedf.Cuisine.unique()

In [ ]:
pprint(recipedf["Cuisine"].value_counts().to_string())

In [ ]:
cudict = {'Fusion' :('World','Fusion','All'),'Indian':('Indian','All','All'),
          'Maharashtrian':('Indian','West','Maharashtrian'),'Punjabi':('Indian','North','Punjabi'),
          'Italian':('World','Italian','All'),'Chinese':('World','Chinese','All'),
          'American':('World','American','All'),'Gujarati':('Indian','West','Gujarati'),
          'Thai':('World','Thai','All'),'Hyderabadi':('Indian','South','Hyderabadi'),
          'British':('World','British','All'),'French':('World','French','All'),
          'Bengali':('Indian','East','Bengali'),'Kerala':('Indian','South','Kerala'),
          'Karnataka':('Indian','South','Karnataka'),'Uttar Pradesh':('Indian','North','Uttar Pradesh'),
          'Mexican':('World','Mexican','All'),'Tamil Nadu':('Indian','South','Tamil Nadu'),
          'Rajasthani':('Indian','North','Rajasthani'),'Indo-Chinese':('World','Indo Chinese','All'),
          'Goan':('Indian','West','Goan'),'Parsi':('World','Parsi','All'),
          'Middle Eastern':('World','Middle Eastern','All'),'Sindhi':('World','Sindhi','All'),
          'TamilNadu':('Indian','South','Tamil Nadu'),'Andhra':('Indian','South','Andhra'),
          'Kashmiri':('Indian','North','Kashmiri'),'South Indian':('Indian','South','All'),
          'Indian, Fusion':('Indian/World','Fusion','All'),'Spanish':('World','Spanish','All'),
          'Greek':('World','Greek','All'),'Turkish':('World','Turkish','All'),
          'Lebanese':('World','Lebanese','All'),'Other Cuisines':('World/Indian','All','All'),
          'Indo-Italian':('World/Indian','Indo Italian','All'),'Sri Lankan':('World','Sri Lankan','All'),
          'Japanese':('World','Japanese','All'),'Moroccan':('World','Moroccan','All'),
          'Other':('World/Indian','All','All'),'Chettinad':('Indian','South','Tamil Nadu'),
          'Indonesian':('World','Indonesian','All'),'Marwari':('Indian','North','Marwadi'),
          'South Indian, Indian':('Indian','South','All'),'Peruvian':('World','Peruvian','All'),
          'Madhya Pradesh':('Indian','North','Madhya Pradesh'),'Continental':('World','Continental','All'),
          'Oriental':('World','Chinese','All'),'German':('World','German','All'),
          'Vietnamese':('World','Vietnamese','All'),'Mangalorean':('Indian','South','Mangalorean'),
          'Bihari':('Indian','East','Bihari'),'Bengali, Indian':('Indian','East','Bengali'),
          'Indian, Maharashtrian':('Indian','West','Maharashtrian'),'Punjabi, Indian':('Indian','North','Punjabi'),
          'Indian, Punjabi':('Indian','North','Punjabi'),'Singaporean':('World','Singaporean','All'),
          'Malaysian':('World','Malaysian','All'),'Korean':('World','Korean','All'),
          'Maharashtrian, Indian':('Indian','West','Maharashtrian'),'South African':('World','South African','All'),
          'Persian':('World','Parsi','All'),'Mumbai':('Indian','West','Maharashtrian'),
          'Gujarati,  Maharashtrian':('Indian','West','Gujarati/Maharashtrian'),
          'American,  British':('World','American/British','All'),'Other cuisines':('World/Indian','All','All'),
          'Canadian':('World','Canadian','All'),'fusion':('World','Fusion','All'),
          'Moghlai':('Indian','Mughlai','All'),'Indo-Chinese, Fusion':('World','Indo-Chinese/Fusion'),
          'Kerala,  Karnataka,  Tamil Nadu':('Indian','South','Kerala/Karnataka/Tamil Nadu'),
          'Burmese':('World','Burmese','All'),'Swiss':('World','Swiss','All'),
          'Gujarati, Indian':('Indian','West','Gujarati'),'Pakistani':('World','Pakistani','All'),
          'Australian':('World','Australian','All'),'Georgian':('World','Fusion','All'),
          'Portuguese':('World','Portuguese','All'),'Odia':('Indian','East','Odisha'),
          'Kerala,  Tamil Nadu':('Indian','South','Kerala/Tamil Nadu'),'Italian, Fusion':('World','Fusion/Italian'),
          'Punjabi, Indian, Fusion':('Indian','North','Punjabi'),'Orissa':('Indian','East','Odisha'),
          'Tamil Nadu,  Kerala':('Indian','South','Tamil Nadu/Kerala'),'Caribbean':('World','Carribean','All'),
          'Polish':('World','Polish','All'),'French, Fusion':('World','Fusion/French'),
          'Fusion, Indian':('World/Indian','Fusion','All'),'indian':('Indian','All','All'),
          'Cuban':('World','Cuban','All'),'Chettinaad':('Indian','South','Tamil Nadu'),                                       
          'Awadhi':('Indian','North','Awadhi'),'Indian, Bengali':('Indian','East','Bengali'),
          'South Indian, Indian, Fusion':('Indian','South','All'),
          'Maharashtrian, Indian, Fusion':('Indian','West','Maharashtrian'),
          'Indo-Mexican':('World','Fusion','All'),'Indian, Rajasthani':('Indian','North','Rajasthani'),
          'French,  British':('World','French'),'Indo-chinese':('World','Indo Chinese','All'),
          'Indian,  Fusion':('Indian','Fusion','All'),'Scottish':('World','Scottish','All'),
          'Mumbai, Indian':('Indian','West','Maharashtrian'),'Indian, Gujarati':('Indian','West','Gujarati'),
          'Jamaican':('World','Jamaican','All'),'Tibetian':('World','Tibetian','All'),
          'Brazilian':('World','Brazilian','All'),'Karnataka, Indian':('Indian','South','Karnataka'),
          'Russian':('World','Russian','All'),'Chinese, Fusion':('World','Chinese/Fusion','All'),
          'Indian, Parsi':('World','Parsi','All'),'Indian, Hyderabadi':('Indian','South','Hyderabadi'),
          'Tamil Nadu,  Kerala,  Karnataka':('Indian','South','Kerala/Karnataka/Tamil Nadu'),
          'Kerala, Indian':('Indian','South','Kerala'),'Tamil Nadu,  Karnataka':('Indian','South','Karnataka/Tamil Nadu'),
          'Denmark':('World','Denmark','All'),'Sindhi, Indian, Fusion':('World','Sindhi','All'),
          'Rajasthani, Indian':('Indian','North','Rajasthani'),'Goan, Indian':('Indian','West','Goan'),
          'Indian Fusion':('Indian','Fusion','All'),'Indian, Mumbai':('Indian','West','Maharashtrian'),                                    
          'Jain':('Indian','All','All'),'German Cuisine':('World','German','All'),                                   
          'Indonesian,  Malaysian,  Thai':('World','Indonesian/Malaysian/Thai','All'),
          'Fusion, American':('World','America/Fusion','All'),'Kashmiri, Indian':('Indian','North','Kashmiri'),
          'Fusion, French':('World','French/Fusion','All'),'Assam':('Indian','East','Assam'),
          'Chinese, Indo-Chinese':('World','Chinese','All'),'Indian, Fusion, South Indian':('Indian','South','All'),                 
          'Rome':('World','Rome','All'),                                           
          'Mongolian':('World','Mongolian','All'),                                         
          'Indian, South Indian':('Indian','South','All'),                              
          'Spanish,  Italian':('World','Spanish/Italian','All'),                                 
          'Viennese':('World','Viennese','All'),                                          
          'Parsi, Indian':('World','Parsi','All'),                                     
          'Oriental, Fusion':('Indian','Chinese','All'),                                  
          'Indian,  Sri Lankan':('World','Sri Lankan','All'),                               
          'Irish':('World','Irish','All'),                                             
          'Indian, Punjabi, Fusion':('Indian','North','Punjabi'),                           
          'African':('World','African','All'),                                           
          'Tex-Mex':('World','Fusion','All'),                                           
          'North Eastern':('World','East','All'),                                     
          'Mangalorean, Indian':('Indian','South','Mangalorean'),                               
          'Bengali, Fusion':('Indian','Bengali/Fusion','All'),                                   
          'Malvani':('World','Malvani','All'),                                         
          'Maharashtrian,  Gujarati':('Indian','West','Gujarati/Maharashtrian'),                          
          'Indian, Kerala':('Indian','South','Kerala'),                                  
          'Uttar Pradesh, Indian':('Indian','North','Uttar Pradesh'),                             
          'Uttar Pradesh, Punjabi':('Indian','North','Uttar Pradesh/Punjabi'),                          
          'Andhra,  Kerala':('Indian','South','Andhra/Kerala'),                                 
          'Malaysian,  Singaporean':('World','Malaysian/Singaporean','All'),                         
          'Kerala,  Karnataka':('Indian','South','Kerala/Karnataka'),                                
          'Punjabi, Maharashtrian, Karnataka':('Indian','North/West/South','Punjabi/Maharashtrian/Karnataka'),                 
          'Indonesian,  Malaysian':('World','Indonesian/Malaysian','All'),                            
          'Indo-Italian, Fusion':('World','Fusion','All'),                              
          'Korean, Fusion':('World','Korean/Fusion','All'),                                    
          'Italian, British, Fusion':('World','Italian/British/Fusion','All'),                          
          'Punjabi , Indian':('Indian','North','Punjabi'),                                  
          'Maharashtrian,  Tamil Nadu,  Karnataka':('Indian','West/South','Maharashtrian/Tamil Nadu/Karnataka'),            
          'Indian, Mangalorean':('Indian','South','Mangalorean'),                               
          'Lebanese,  Middle Eastern':('Indian','Lebanese/Middle Eastern','All'),                         
          'Goan, Fusion, Indian':('Indian','West','Goan'),                              
          'Maharashtrian,  Karnataka,  Goan':('Indian','West/South','Maharashtrian/Goan/Karnataka'),            
          'Gujarati, Indian, Fusion':('Indian','West','Gujarati'),                          
          'Sindhi, Indian':('World','Sindhi','All'),                                    
          'Coorg':('Indian','South','Karnataka'),                                           
          'Indian, Karnataka':('Indian','South','Karnataka'),                               
          'karnataka':('Indian','South','Karnataka'),                                         
          'Indo Chinese, Indo-Chinese':('World','Indo Chinese','All'),                      
          'Indo Chinese':('World','Indo Chinese','All'),                                      
          'Thai,  Malaysian,  Singaporean,  Indonesian':('World','Thai/Malaysian/Singaporean/Indonesian','All'),       
          'Indian, Kashmiri':('Indian','North','Kashmiri'),                                  
          'Mediterranean':('World','Middle Eastern','All'),                                     
          'Punjabi,  Hyderabadi':('Indian','North/South','Punjabi/Hyderabadi'),                            
          'British, Fusion':('World','British/Fusion','All'),                                   
          'Phillipine':('World','Fusion','All'),                                        
          'Indian, Uttar Pradesh':('Indian','North','Uttar Pradesh'),                   
          'Gujarati, Fusion':('Indian','West','Gujarati'),                                  
          'Middle Eastern, Fusion':('World','Middle Eastern/Fusion','All'),                            
          'Creole':('World','American','All'),                                            
          'parsi':('World','Parsi','All'),                                             
          'Italian,  Spanish':('World','Italian/Spanish','All'),                                
         }

In [ ]:
for i in range(len(recipedf)):
    recipedf.iloc[i][2] = cudict[recipedf.iloc[i][2]]

In [ ]:
recipedf

In [ ]:
pickle.dump(recipedf,open("recipedfmid.p","wb"))

In [ ]:
recipedf

# Ingredients (From here its ingredient stuff that we have tried and changed and stuff till the end)

In [ ]:
recipedfmid = pickle.load(open("recipedfmid.p","rb"))

In [ ]:
pickle.dump(recipedf,open("recipedfmid.p","wb"))

In [49]:
import pickle

In [50]:
recipedfmid = pickle.load(open("recipedfmid.p","rb"))

In [51]:
recipedfmid

,URL,Dish,Cuisine,TotalTime,Ingredients,Style
0,https://www.sanjeevkapoor.com/Recipe/Samosa-Sa...,Samosa Sandwich,"(Indian, All, All)","(0, 0, 22)","[Samosas 4 , White bread slices 8 , Butt...",Veg
1,https://www.sanjeevkapoor.com/Recipe/Sattu-ki-...,Sattu ki Kachori,"(Indian, All, All)","(0, 0, 47)","[Dough , Refined flour (maida) 1 1/2 cups , ...",Veg
2,https://www.sanjeevkapoor.com/Recipe/Rawa-Toas...,Rawa Toast,"(World, Fusion, All)","(0, 0, 42)","[Semolina (rawa/suji) 100 grams , Bread slice...",Veg
3,https://www.sanjeevkapoor.com/Recipe/Rasam-Vad...,Rasam Vada,"(Indian, South, All)","(0, 0, 57)",[Split pigeon pea (toor dal/arhar dal) soaked ...,Veg
4,https://www.sanjeevkapoor.com/Recipe/Rajma-Pul...,Rajma Pulao,"(Indian, All, All)","(0, 7, 26)",[Red kidney beans (rajma) (soaked overnight an...,Veg
...,...,...,...,...,...,...
12179,https://www.sanjeevkapoor.com/Recipe/Kozhi-Var...,Kozhi Vartha Kari,"(Indian, South, Kerala)","(0, 0, 57)","[Chicken cut into 16 pieces 1 kilogram , Drie...",Non-Veg
12180,https://www.sanjeevkapoor.com/Recipe/Besan-ke-...,Besan ke Laddoo,"(Indian, West, Maharashtrian)","(0, 0, 42)","[Coarse gram flour (besan) 2 cups , Cashewnut...",Veg
12181,https://www.sanjeevkapoor.com/Recipe/Besan-ki-...,Besan ki Burfi,"(Indian, North, Uttar Pradesh)","(0, 0, 57)","[Gram flour (besan) 2 cups , Green cardamom p...",Veg
12182,https://www.sanjeevkapoor.com/Recipe/Chocolate...,Chocolate Burfi,"(Indian, All, All)","(0, 0, 42)","[Cocoa powder 2 teaspoons , Khoya/mawa 2 cup...",Veg


In [52]:
def firstremoval(a):
    final = []
    weird = ['â¾','â½','â¼']
    for s in a:
        s = s.lower()
        s = s.replace(',', '')
        for i in weird:
            s = s.replace(i,'')
        
        k = ''
        if s == "  ":
            continue
        
        if s.split()[0][0].isnumeric():
            k = " ".join(s.split()[1:])
            
        else:
            for w in s.split():
                if(w[0].isdigit()):
                    break
                k+= w + " "
        
        final.append(k.strip())
        
    return final

In [53]:
for i in range(len(recipedfmid)):
    
    k = recipedfmid.iloc[i]["Ingredients"]
    recipedfmid.iloc[i]["Ingredients"] = firstremoval(k)
#     except:
#         print(i)
    

In [54]:
def secondremoval(a):
    words = ['as','for','to','into','sliced','chopped','finely','cubed','medium',
             'peeled','cut','blanched','boiled','diced','shelled','deseeded',
             'seeded','roughly','mixed','whisked','roasted','fried','pitted',
             'halved','parboiled','rubbed','soaked','dried','quartered','tablespoon','cup','skinned',
             'toasted','soaked','teaspoon','fine','blended','thickly','tinned','canned','stemmed',
             'blanvhed','grated','a','boiled,','thinly','shredded','cooked','slivered',
             'sprinkling','pickled','diagonally','washed','trimmed','handful','freshly','filtered'
             'pulp','slit','salted','cleaned','crushed','(soaked','coarsely','roundels','smoked',
             'chilled','drained','beaten','inch','julienne','strips','thickly','triangles','thin']
    final = []
    for s in a:
        k = s.split()[0] + " "
        for w in s.split()[1:]:
            if(w in words):
                break
            k+= w + " "
        final.append(k.strip())
    return final

In [55]:
for i in range(len(recipedfmid)):
    try:
        k = recipedfmid.iloc[i]["Ingredients"]
        recipedfmid.iloc[i]["Ingredients"] = secondremoval(k)
    except:
        print(i)

In [56]:
blacklist = ['a ','curly ','dollop ','few ','drops ','of ','sprigs ','thin ' ,'strips ','fresh ','generous ', 'pinch ',
             'large ','+','little ','sprig ',' pieces',' slices','roasted ','chopped ','slivered ',
             'small ','medium ','smoked ','soaked ','zest ','cooked ','teas']

In [57]:
for i in range(len(recipedfmid)):
    for j in range(len(recipedfmid.iloc[i][4])):
        for w in blacklist:
            if(w in recipedfmid.iloc[i][4][j]):
                recipedfmid.iloc[i][4][j]  = recipedfmid.iloc[i][4][j].replace(w,' ').strip()

In [58]:
for i in range(len(recipedfmid)):
    for j in range(len(recipedfmid.iloc[i][4])):
        recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].lower()
        if('yellow pepper' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('yellow pepper','bell pepper')
        if('soy chunks' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy chunks','soy bean')
        if('soy dip' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy dip','soy bean')
        if('soy flour' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy flour','soy bean')
        if('soy granuals' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy granuals','soy bean')
        if('soy granules' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy granules','soy bean')
        if('soy granules (keema)' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy granules (keema)','soy bean')
        if('soy granules ground' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy granules ground','soy bean')
        if('soy mince' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy mince','soy bean')
        if('soy nuggets' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('soy nuggets','soy bean')
        if('refined flour' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('refined flour', 'all purpose flour')
        if('fresh coriander' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('fresh coriander','coriander leaves')
        if('fresh mint' in recipedfmid.iloc[i][4][j] ):
            recipedfmid.iloc[i][4][j]=recipedfmid.iloc[i][4][j].replace('fresh mint','mint leaves')


In [59]:
vocabnew=[
    'Salt/Namak','Water/Paani','Oil/Tel','Turmeric Powder/Haldi','Ginger/Adrak','Asafoetida/Hing',
    'Coriander Leaves/Dhania Patta','Cumin Seeds/Jeera','Curry Leaves/Kadhi Patta','Red Chilli Powder/Lal Mirch',
    'Green Chilli/Hari Mirch','Mustard Seeds/Rai','Onion/Pyaaz','Tomato/Tamatar','Lemon Juice/Lime Juice/Nimbu Ras','Cumin Powder/Jeera Powder',
    'Clove/Laung','Cinnamon/Dal Cheeni','Garlic/Lahsun','Green Cardamom/Choti Elaichi',
    'Coriander Powder/Dhania Powder','Coconut/Nariyal','Potato/Aloo','Indian Bay Leaf/Tej Patta','Cashew/Kaju','Red Chilli/Lal Mirch','Black Pepper Powder/Kaali Mirch Powder',
    'Baking Soda','Chaat Masala','Clarified Butter/Ghee','Chickpea Flour/Besan','Mint Leaves/Pudina Patta','Peanuts/Moong Fali','Wheat Flour/Atta',
    'Rice Flour/Chawal Atta','Carrot/Gajar','Coriander Seeds/Dhania Beej','Jaggery/Gur','Black Salt/Kala Namak','Coconut Oil/Nariyal Tel','Mango Powder/Amchur Powder','Saffron/Kesar',
    'Cardamom Powder/Elaichi Powder','Black Cardamom/Badi Elaichi','Carrom Seeds/Ajwain','Fennel Seeds/Saunf','Peas/Matar','Black Gram/Urad Dal','Basmati Rice/Basmati Chawal','Fenugreek Leaves/Kasuri Methi',
    'Cilantro Leaves/Dhania Patta','Tamarind/Imli','Sesame Seeds/Til','Mustard Oil/Sarso Tel','Beans/Gwar Fali','Mace/Javitri','Almond/Badam','Celery/Ajwain Patta','Cottage Cheese/Paneer','Milk/Doodh','Baking Powder','All Purpose Flour/Maida','Butter/Margarine/Makhan','Ice/Baraf',
    'Fenugreek Seeds/Methi','Curd/Yogurt/Dahi','Cream/Malai','Cabbage/Patta Gobi','Raisins/KishMish','Tapioca Pearls/Sabudana','Olive Oil','Soy Sauce','Sesame Oil/Til Tel','Cauliflower/Gobi','Bengal Gram/Chana Dal',
    'Rice/Chawal','Spring Onion/Hara Pyaaz','Garam Masala','Coconut Milk/Nariyal Doodh','Bread','Peanut Oil/Moong Fali Tel','Sambar Powder','Capsicum/Bell Pepper/Shimla Mirch','Mushroom','Dry Ginger Powder/Saunth','Nutmeg/Jaifal','Mango/Aam',
    'Black Peppercorns/Kali Mirch','Poppy Seeds/Khus Khus','Cucumber/Kheera/Kakdi','Mustard/Sarso','Pistachios/Pista','Lemon/Nimbu','Okra/LadyFinger/Bhindi','Ginger Garlic Paste','Sev','Spinach/Palak','Star Anise/Chakriphool',
    'Sugar/Cheeni','Tomato Puree','Banana/Kela','Oregano','Pigeon Pea Lentils/Arhar Dal/Tuvar Dal','Vegetable Stock','Green Gram/Moong Dal','Pomegranate/Anaar','Flattened Rice/Poha','Caraway Seeds/Shahi Jeera','Vanilla Extract','Baby Corn','Yeast','Corn Starch/Corn Flour',
    'Pomegranate Seeds/Anardana','Strawberry','Rose Water/Gulab Jal','Nigella Seeds/Kalonji','Vinegar/Sirka','Piece Stone Flower/Dagad Phool','Sunflower Oil','White Pepper/Safed Mirch','Basil Leaves/Tulsi',
    'Water Chestnut Flour/Paniphal Flour/Singhare Ka Atta','Green Chutney/Coriander Chutney','Buckwheat Flour/Kuttu Atta','Cheese','Dates/Khajoor','Walnut/Akhrot','Tomato Ketchup/Sauce','Bread Crumbs','Corn/Makki','Beetroot/Beets/Chukandar','Dill Leaves/Savaa Patta','Pumpkin/Kaddu','Rose Syrup','Yam/Jimikand/Suran/Garadu',
    'Sunflower Seeds/Surajmukhi Beej','Kapok Buds/Marathi Moggu','Kokum','Galangal/Kulanjan','Sweet Potatoes/Shakarkandi','Indian Gooseberry/Amla','Ash Gourd/Petha','Pav Bhaji Masala','Puffed Rice/Murmura','Red Chilli Flakes','Rose Petals/Gulab','Colocasia Leaves/Arbi Patta','Idli Rice','Cocoa Powder',
    'Semolina/Rawa/Sooji','Apple/Seb','Noodles','Fruit Salt/Eno','Lettuce','Idli Batter/Uttapam Batter','Raw Banana','Sprouted Moong Beans/Sprouts','Lemon Grass','White Chickpeas/Chole/Kabuli Chana/Safed Chana','Turmeric Leaves','Papdis',
    'Papaya/Papita','Pineapple/Ananas','Black Chickpeas/Kala Chana','Amaranth Flour/Rajgira Atta','Camphor/Kapoor','Indian Five Spice/Panch Phoron','Orange Juice/Santra Ras','Tutti Frutti','Sichuan Pepper','Coconut Chutney','Bitter Gourd/Karela',
    'Pink Lentils/Masoor Dal','Schezwan Sauce','Almond Milk','Millet Flour/Bajra Flour','Fig/Anjeer','Dry Fruits','Finger Millet Flour/Ragi Flour','Buttermilk/Chaas','Kaffir Lime','Parsley','Barnyard Millet/Sama Ke Chawal','Arrow Root Flour',
    'Brinjal/Eggplant/Baingan','Melon Seeds/Magaz','White Cowpeas/Red Cowpeas/Lobia','Lilva Beans/Hyacinth Beans/Avrekaalu','Goda Masala/Kala Masala','Colocasia Roots/Arbi','Honey/Shahad','Milk Powder','Seeraga Samba Rice','Coffee','Orange/Santra',
    'Bottle Gourd/Opo Squash/Lauki/Dudhi','Sodium Benzoate','Radish/Mooli','Chenopodium/Bathua Leaves','Saag','Maize Flour/Makki Atta','Soy Milk','Custard Powder','Khoya/Mawa','Almond Meal/Almond Paste','Flax Seeds/San Beej','Soyabean','Burger Buns','Oats','Aam Panna',
    'Sorghum Flour/Jowar Flour','Foxnuts/Lotus Seeds/Makhana','Jalapeno','Raw Mango/Kairi/Kachcha Aam','Drumstick','Kidney Beans/Rajma','Guava/Amrood','Ivy Gourds/Tendli','Tofu','Avocado','Vermicelli','Akki Shavige','Sapota/Chickoo',
    'Sandalwood Powder','Boondi','Vangi Bath Masala Powder','Smoked Paprika','Sriracha Sauce','Broccoli','Watermelon/Tarbooz','Black Peas/Kala Vatana','Tea/Chai','Agar Agar','Bengal Quince/Stone Apple/Wood Apple/Bel Fruit','Mayonnaise','Snake Gourd/Padwal/Chichinda',
    'Blueberry','Pani Puris/Golgappa Puris','Cherry','Chia Seeds','Neem','Ridge Gourd/Turai','Idli Podi','Pav','Condensed Milk','Basil Seeds/Sabja','Jackfruit/Katahal','Pointed Gourds/Parwal','Bread Croutons','Broken Wheat/Dalia','Chocolate','Marshmallow','Biscuit','Papad','Chicken','Fish','Mutton','Crab','Egg/Anda','Chive','Chipotle','Cuddapah Almond/Chironji',
    'Lychee','Apricot','Peach','Zucchini/Courgette','Zatar','Squash','Worcestershire Sauce','Thai Curry Paste','Yellow Chilli','Wonton Wrapper','Pasta','Spaghetti','Pizza Base','White Wine','Quinoa','Grape/Grapes','Whiskey','Wasabi','Vodka','Gelatin','Turnip','Tuna','Tahini','Treacle','Tobasco Sauce','Prawn',
    'Thyme','Tequila','Chhole Masala','Meat Masala','Tarragon','Olives','Prunes','Star Fruit','Squid','Soy Bean','Soup Stick','Sour Dough Starter','Shrimp','Lamb','Sherry','Screwpine','Scallop','Sausage','Salsa','Salmon','Sake','Sage','Rum','Rosemary','Red Wine','Raspberry','Rasam Powder','Quail','Pork','Pomfret','Plum',
    'Pine Nuts/Chilgoza','Gherkin','Peri Peri Masala','Pear','Passion Fruit','Oyster Sauce','Oatmeal','Nihari Masala','Nachos','Mussels','Muskmelon','Mussallam Masala','Mulberry','Muesli','Moringa Leaves','Molasses','Gun Powder/Molgapodi','Jam','Lotus Stem','Long Pepper/Pipli','Lobseter','Leek','Green Chana',
    'Kiwi','Kale','Indian Blackberry/Jamun','HP Sauce','Hazelnut','Ham','Gum/Gond','Maple Syrup','Cranberry','Couscous','Clams','Sapota/Chikoo','Biryani Masala','Beer','Barley','Barbeque Sauce','Bacon','Asparagus','Artichoke','Aioli','Alfalfa','Bok Choy','Duck','Shallot','Tapioca Flour','Amaranth'
    
]

In [60]:
for i in range(len(vocabnew)):
    vocabnew[i]=vocabnew[i].lower()

In [61]:
corr = {'mustad':'Mustard',

        'capsicu':'Bell Pepper',
        'wonton':'Wonton Wrapper',
        'wonton':'Wonton Wrapper',
        'wholw':'Whole',
        'wholewheat':'Whole Wheat',
        'whole-wheat':'Whole Wheat',
        'pizz':'Pizza',
        'four':'Flour',
        'buns': 'Burger Buns',
        'buger': 'Burger',
        '(gehun)':'Whole Wheat Flour',
        'cashewnuts':'Cashew',
        'raddish': 'radish',
        'seasame' :'sesame',
        'whisky': 'whiskey',
        'vanill': 'vanilla',
        'coco': 'cocoa',
        'fruity': 'fruitti',
        'turmuric': 'turmeric',
        'tun': 'tuna',
        'toovar':'tuvar',
        'toor': 'tuvar',
        'toamtoes':'tomato',
        'cherries':'cherry',
        'tindli':'tendli',
        'thymecrushed':'thyme',
        'thymefinely':'thyme',
        'pe':'pea',
        'te':'tea',
        'masal':'masala',
        'tapioc':'tapioca',
        'tamrind':'tamarind',
        'tamirind':'tamarind',
        'tamarin':'tamarind',
        'god': 'goda',
        'suger':'sugar',
        'sugsar':'sugar',
        'subz':'sabja',
        'subza':'sabja',
        'sabz':'sabja',
        'dagadphool': 'dagad phool',
        'strawberries':'strawberry',
        'srirach':'sriracha',
        'srimps':'shrimps',
        'sprouted':'sprouts',
        'spring': 'spring onion',
        'piegeon':'pigeon',

        '(jowar)':'sorghum flour',
        'shrimpspeeled': 'shrimp',
        'semolin':'semolina',
        'semoina':'semolina',
        'seasame':'sesame',
        'ol': 'oil',
        'sambhar':'sambar',
        'sambariy':'sambar',
        'schezwan': 'schezwan sauce',
        'roses':'gulab',

        'refind':'refined',
        'chili':'chilli',
        'chiili':'chilli',
        'raspberries':'raspberry',
        'rasberry':'raspberry',
        'rasins' : 'raisins',
        'rasam' : 'rasam powder',
        'rajm' : 'rajma',
        'rajir':'rajgira',
        'rajgire': 'rajgira',
        'att':'atta',
        'aata' : 'atta',
        'rahu' : 'rahu fish',
        'raddish' : 'radish',
        'quino' : 'quinoa',
        'golgapp':'golgappa',
        'praws' : 'prawns',
        'potaoes':'potatoes',
        'potaotes':'potatoes',
        'poato': 'potatoes',
        'sauages' : 'sausages',
        'pomogranate' : 'pomegranate',
        'pok': 'bok',
        'pak':'bok',
        'pok': 'bok',
        'choi':'choy',
        'bokchoy': 'bok choy',
        'pinenuts' : 'pine nuts',
        'periperi' : 'peri peri',
        'penuts' : 'peanuts',
        'parwar': 'parwal',
        'praval' : 'parwal',
        'paprika' : 'smoked paprika',
        'papay' : 'papaya',
        'paanch' : 'panch',
        'paanch' : 'panch',
        'onoin': 'onion',
        'oinion' : 'onion',
        'nutmet' : 'nutmeg',
        'nacho' : 'nachos',
        'mutoon' : 'mutton',
        'miutton' : 'mutton',
        'motton' : 'mutton',
        'musterd': 'mustard',
        'mstard' : 'mustard',
        'mulberries' : 'mulberry',
        'mozzrella' : 'mozzarella',
        'mozzarell': 'mozzarella',
        'moring' : 'moringa',
        'molagapodi' : 'molgapodi',
        'mkayonnaise' : 'mayonnaise',
        'chewda' : 'poha',
        'marshmellow' : 'marshmallow',
        'kootoo' : 'kuttu',
        'lasun' : 'lahsun',
        'jawari' : 'jowar',
        'jaggey': 'jaggery',
        'jagger' : 'jaggery',
        'coffe' : 'coffee',
        'guav' : 'guava',
        'gren' : 'green',
        'cradamoms': 'cardamom',
        'cardamon': 'cardamom' ,
        'caramom' : 'cardamom',
        'chtuney': 'chutney',
        'chuntney' : 'chutney',
        'chlli' : 'chilli',
        'god' : 'goda',
        'garli' : 'garlic',
        'funegreek':'fenugreek',
        'besil' : 'basil',
        'drumbstick' : 'drumstick',
        'dil' : 'dill',
        'cocontu': 'coconut',
        'cconut' : 'coconut',
        'cococnut': 'coconut',
        'cocnut': 'coconut',
        'cumi' : 'cumin',
        'bleaves' : 'leaves',
        'chese' : 'cheese',
        'corinader': 'coriander',
        'coriabder'  : 'coriander',
        'colocasi': 'colocasia' ,
        'colocassi' : 'colocasia',
        'colocassia': 'colocasia',
        'coco' : 'cocoa',
        'clabbage' :'cabbage',
        'chikkoos' : 'chikoo',
        'chiciken': 'chicken' ,
        'chicke': 'chicken', 
        'chcken': 'chicken' , 
        'chcicken' : 'chicken',
        'cauiliflower' : 'cauliflower',
        'caulfilower': 'cauliflower',
        'carrrots': 'carrots',
        'capsico' : 'capsicum',
        'cahew' : 'cashew',
        'brijals': 'brinjal',
        'borccoli' : 'broccoli',
        'better gourds' : 'bitter gourd',
        'barbecue' : 'barbeque',
        'banan' : 'banana',
        'sod': 'soda',
        'soada': 'soda',
        'bajr'  : 'bajra',
        'asafoetid': 'asafoetida' ,
        'asafeotida': 'asafoetida',
        'asafoitida' : 'asafoetida',
        'amonds' : 'almonds',
       'chenna':'Paneer',
       'sprig':'Leaves',
       'cranberries':'Cranberry',
       'dried':'Dry',
       'bean':'beans',
       'prune':'prunes',
       'wontons':'wonton',
        'ladyfiners':'ladyfinger',
        'macaroni':'Pasta',
        'farfalle':'Pasta',
        'mackerel':'fish',
        'chillli':'chilli',
        'chiullies':'chilli',
        'avacado':'avocado'
       }

In [71]:
for i in range(len(recipedfmid)):
    for j in range(len(recipedfmid.iloc[i][4])):
        s = recipedfmid.iloc[i][4][j].split()
        for k in s:
            if k in corr:
                ind=s.index(k)
                s.remove(k)
                s.insert(ind,corr[k])
        recipedfmid.iloc[i][4][j] = recipedfmid.iloc[i][4][j].replace(recipedfmid.iloc[i][4][j],' '.join(s))
        

In [88]:
recipedfmid=recipedfmid.rename(columns={'TotalTime':'Total Time'})


In [89]:
recipedfmid

,URL,Dish,Cuisine,Total Time,Ingredients,Style
0,https://www.sanjeevkapoor.com/Recipe/Samosa-Sa...,Samosa Sandwich,"(Indian, All, All)","(0, 0, 22)","[samosas, white bread, butter, green chutney, ...",Veg
1,https://www.sanjeevkapoor.com/Recipe/Sattu-ki-...,Sattu ki Kachori,"(Indian, All, All)","(0, 0, 47)","[dough, all purpose flour (maida), carom seeds...",Veg
2,https://www.sanjeevkapoor.com/Recipe/Rawa-Toas...,Rawa Toast,"(World, Fusion, All)","(0, 0, 42)","[semolina (rawa/suji), bread, salt, yogurt, on...",Veg
3,https://www.sanjeevkapoor.com/Recipe/Rasam-Vad...,Rasam Vada,"(Indian, South, All)","(0, 0, 57)","[split pigeon pea (toor dal/arhar dal), urad d...",Veg
4,https://www.sanjeevkapoor.com/Recipe/Rajma-Pul...,Rajma Pulao,"(Indian, All, All)","(0, 7, 26)","[red kidney beans (rajma), rice, ghee, black p...",Veg
...,...,...,...,...,...,...
12179,https://www.sanjeevkapoor.com/Recipe/Kozhi-Var...,Kozhi Vartha Kari,"(Indian, South, Kerala)","(0, 0, 57)","[chicken, Dry red chillies, ginger, garlic, re...",Non-Veg
12180,https://www.sanjeevkapoor.com/Recipe/Besan-ke-...,Besan ke Laddoo,"(Indian, West, Maharashtrian)","(0, 0, 42)","[coarse gram flour (besan), Cashew, almonds, g...",Veg
12181,https://www.sanjeevkapoor.com/Recipe/Besan-ki-...,Besan ki Burfi,"(Indian, North, Uttar Pradesh)","(0, 0, 57)","[gram flour (besan), green cardamom powder, pi...",Veg
12182,https://www.sanjeevkapoor.com/Recipe/Chocolate...,Chocolate Burfi,"(Indian, All, All)","(0, 0, 42)","[cocoa powder, khoya/mawa, sugar, liquid gluco...",Veg


In [80]:
# finalpd = pickle.load(open("finalpd.p","rb"))
infile = open('finalpd','rb')
new_dict = pickle.load(infile)
infile.close()

In [81]:
new_dict

,URL,Dish,Cuisine,Total Time,Ingredients,Style
0,https://www.vegrecipesofindia.com/eggless-date...,Easy Date And Walnut Cake (Eggless + Vegan),"(World, All, All)","(0, 1, 20)","[Baking Powder, Water, Whole Wheat Flour, Vani...",Vegan
1,https://www.vegrecipesofindia.com/eggless-oran...,"Orange Cake (Whole Wheat, Eggless & Vegan)","(World, All, All)","(0, 0, 55)","[Regular Fine Sugar, Olive Oil, Orange Juice, ...",Vegan
2,https://www.vegrecipesofindia.com/eggless-bana...,Whole Wheat Banana Cake (Eggless And Vegan),"(World, American, All)","(0, 0, 40)","[Baking Powder, Baking Soda, Whole Wheat Flour...",Vegan
3,https://www.vegrecipesofindia.com/punjabi-chol...,Chana Masala | Punjabi Chole Masala (Stovetop ...,"(Indian, North, All)","(0, 9, 45)","[Cumin Seeds, Water / The Stock In Which The C...",Vegan
4,https://www.vegrecipesofindia.com/idli-recipe-...,Idli Recipe | How To Make Idli Batter,"(Indian, South, All)","(0, 9, 25)","[Thick Poha / Thick Poha Flattened Rice, Oil A...",Vegan
...,...,...,...,...,...,...
1650,https://www.vegrecipesofindia.com/rishi-pancha...,Rishi Panchami Bhaji Recipe | Rushichi Bhaji R...,"(Indian, West, Konkani & Malvani/Maharashtrian)","(0, 1, 0)","[Colocasia Leaf Stems Arbi Ke Danthal, Elephan...",Satvik (No onion-no garlic)
1651,https://www.vegrecipesofindia.com/peanut-rice-...,Peanut Rice,"(Indian, South, Tamil Nadu)","(0, 0, 30)","[Asafoetida, Turmeric Powder, Sesame Oil Ginge...",Satvik (No onion-no garlic)
1652,https://www.vegrecipesofindia.com/pumpkin-samb...,Pumpkin Sambar | Poosanikai Sambar,"(Indian, South, Tamil Nadu)","(0, 0, 40)","[Tuvar Dal Pigeon Pea Lentils / Arhar Dal, Tur...",Satvik (No onion-no garlic)
1653,https://www.vegrecipesofindia.com/podi-idli-re...,Podi Idli,"(Indian, South, Tamil Nadu)","(0, 0, 30)","[Chana Dal Husked Split / Whole Bengal Gram, C...",Satvik (No onion-no garlic)


In [90]:
import pandas as pd
db=pd.concat([new_dict,recipedfmid],sort=False)

In [91]:
db

,URL,Dish,Cuisine,Total Time,Ingredients,Style
0,https://www.vegrecipesofindia.com/eggless-date...,Easy Date And Walnut Cake (Eggless + Vegan),"(World, All, All)","(0, 1, 20)","[Baking Powder, Water, Whole Wheat Flour, Vani...",Vegan
1,https://www.vegrecipesofindia.com/eggless-oran...,"Orange Cake (Whole Wheat, Eggless & Vegan)","(World, All, All)","(0, 0, 55)","[Regular Fine Sugar, Olive Oil, Orange Juice, ...",Vegan
2,https://www.vegrecipesofindia.com/eggless-bana...,Whole Wheat Banana Cake (Eggless And Vegan),"(World, American, All)","(0, 0, 40)","[Baking Powder, Baking Soda, Whole Wheat Flour...",Vegan
3,https://www.vegrecipesofindia.com/punjabi-chol...,Chana Masala | Punjabi Chole Masala (Stovetop ...,"(Indian, North, All)","(0, 9, 45)","[Cumin Seeds, Water / The Stock In Which The C...",Vegan
4,https://www.vegrecipesofindia.com/idli-recipe-...,Idli Recipe | How To Make Idli Batter,"(Indian, South, All)","(0, 9, 25)","[Thick Poha / Thick Poha Flattened Rice, Oil A...",Vegan
...,...,...,...,...,...,...
12179,https://www.sanjeevkapoor.com/Recipe/Kozhi-Var...,Kozhi Vartha Kari,"(Indian, South, Kerala)","(0, 0, 57)","[chicken, Dry red chillies, ginger, garlic, re...",Non-Veg
12180,https://www.sanjeevkapoor.com/Recipe/Besan-ke-...,Besan ke Laddoo,"(Indian, West, Maharashtrian)","(0, 0, 42)","[coarse gram flour (besan), Cashew, almonds, g...",Veg
12181,https://www.sanjeevkapoor.com/Recipe/Besan-ki-...,Besan ki Burfi,"(Indian, North, Uttar Pradesh)","(0, 0, 57)","[gram flour (besan), green cardamom powder, pi...",Veg
12182,https://www.sanjeevkapoor.com/Recipe/Chocolate...,Chocolate Burfi,"(Indian, All, All)","(0, 0, 42)","[cocoa powder, khoya/mawa, sugar, liquid gluco...",Veg


In [92]:
filename = 'db'
outfile = open(filename,'wb')

In [93]:
pickle.dump(db,outfile)
outfile.close()

In [50]:
import pickle
infile = open('db','rb')
db = pickle.load(infile)
infile.close()

In [51]:
db.Style.unique()

array(['Vegan', 'Gluten Free', 'Satvik (No onion-no garlic)', 'Veg',
       'Non-Veg', 'Satvik'], dtype=object)

In [52]:
db=db.reset_index()

In [53]:
db=db.drop('index',axis=1)

In [54]:
db['Style']=db['Style'].replace(to_replace ='Satvik (No onion-no garlic)', value ='Satvik (Jain)')


In [55]:
db['Style']=db['Style'].replace(to_replace ='Satvik', value ='Satvik (Jain)')

In [56]:
db.Style.unique()

array(['Vegan', 'Gluten Free', 'Satvik (Jain)', 'Veg', 'Non-Veg'],
      dtype=object)

In [57]:
db

,URL,Dish,Cuisine,Total Time,Ingredients,Style
0,https://www.vegrecipesofindia.com/eggless-date...,Easy Date And Walnut Cake (Eggless + Vegan),"(World, All, All)","(0, 1, 20)","[Baking Powder, Water, Whole Wheat Flour, Vani...",Vegan
1,https://www.vegrecipesofindia.com/eggless-oran...,"Orange Cake (Whole Wheat, Eggless & Vegan)","(World, All, All)","(0, 0, 55)","[Regular Fine Sugar, Olive Oil, Orange Juice, ...",Vegan
2,https://www.vegrecipesofindia.com/eggless-bana...,Whole Wheat Banana Cake (Eggless And Vegan),"(World, American, All)","(0, 0, 40)","[Baking Powder, Baking Soda, Whole Wheat Flour...",Vegan
3,https://www.vegrecipesofindia.com/punjabi-chol...,Chana Masala | Punjabi Chole Masala (Stovetop ...,"(Indian, North, All)","(0, 9, 45)","[Cumin Seeds, Water / The Stock In Which The C...",Vegan
4,https://www.vegrecipesofindia.com/idli-recipe-...,Idli Recipe | How To Make Idli Batter,"(Indian, South, All)","(0, 9, 25)","[Thick Poha / Thick Poha Flattened Rice, Oil A...",Vegan
...,...,...,...,...,...,...
13834,https://www.sanjeevkapoor.com/Recipe/Kozhi-Var...,Kozhi Vartha Kari,"(Indian, South, Kerala)","(0, 0, 57)","[chicken, Dry red chillies, ginger, garlic, re...",Non-Veg
13835,https://www.sanjeevkapoor.com/Recipe/Besan-ke-...,Besan ke Laddoo,"(Indian, West, Maharashtrian)","(0, 0, 42)","[coarse gram flour (besan), Cashew, almonds, g...",Veg
13836,https://www.sanjeevkapoor.com/Recipe/Besan-ki-...,Besan ki Burfi,"(Indian, North, Uttar Pradesh)","(0, 0, 57)","[gram flour (besan), green cardamom powder, pi...",Veg
13837,https://www.sanjeevkapoor.com/Recipe/Chocolate...,Chocolate Burfi,"(Indian, All, All)","(0, 0, 42)","[cocoa powder, khoya/mawa, sugar, liquid gluco...",Veg


In [58]:
for i in range(len(db)):
    for j in range(len(db.iloc[i][4])):
        db.iloc[i][4][j]=db.iloc[i][4][j].lower()

In [59]:
db

,URL,Dish,Cuisine,Total Time,Ingredients,Style
0,https://www.vegrecipesofindia.com/eggless-date...,Easy Date And Walnut Cake (Eggless + Vegan),"(World, All, All)","(0, 1, 20)","[baking powder, water, whole wheat flour, vani...",Vegan
1,https://www.vegrecipesofindia.com/eggless-oran...,"Orange Cake (Whole Wheat, Eggless & Vegan)","(World, All, All)","(0, 0, 55)","[regular fine sugar, olive oil, orange juice, ...",Vegan
2,https://www.vegrecipesofindia.com/eggless-bana...,Whole Wheat Banana Cake (Eggless And Vegan),"(World, American, All)","(0, 0, 40)","[baking powder, baking soda, whole wheat flour...",Vegan
3,https://www.vegrecipesofindia.com/punjabi-chol...,Chana Masala | Punjabi Chole Masala (Stovetop ...,"(Indian, North, All)","(0, 9, 45)","[cumin seeds, water / the stock in which the c...",Vegan
4,https://www.vegrecipesofindia.com/idli-recipe-...,Idli Recipe | How To Make Idli Batter,"(Indian, South, All)","(0, 9, 25)","[thick poha / thick poha flattened rice, oil a...",Vegan
...,...,...,...,...,...,...
13834,https://www.sanjeevkapoor.com/Recipe/Kozhi-Var...,Kozhi Vartha Kari,"(Indian, South, Kerala)","(0, 0, 57)","[chicken, dry red chillies, ginger, garlic, re...",Non-Veg
13835,https://www.sanjeevkapoor.com/Recipe/Besan-ke-...,Besan ke Laddoo,"(Indian, West, Maharashtrian)","(0, 0, 42)","[coarse gram flour (besan), cashew, almonds, g...",Veg
13836,https://www.sanjeevkapoor.com/Recipe/Besan-ki-...,Besan ki Burfi,"(Indian, North, Uttar Pradesh)","(0, 0, 57)","[gram flour (besan), green cardamom powder, pi...",Veg
13837,https://www.sanjeevkapoor.com/Recipe/Chocolate...,Chocolate Burfi,"(Indian, All, All)","(0, 0, 42)","[cocoa powder, khoya/mawa, sugar, liquid gluco...",Veg


In [62]:
filename = 'db2'
outfile = open(filename,'wb')
pickle.dump(db,outfile)
outfile.close()